In [ ]:
import pandas as pd

In [ ]:
cd '/content/drive/MyDrive/프로젝트2-B팀/요약 관련/'

/content/drive/MyDrive/프로젝트2-B팀/요약 관련


In [ ]:
#여기에는 지원내용이랑 지원대상 컬럼이 없는 것 같아서 그냥 원래 데이터 가져올게용.

data = pd.read_csv("policy_metetable(DSL전달용).csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (23,26,28,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
import numpy as np

In [ ]:
data_1000 = data.sample(n = 10, replace = True)

In [ ]:
exdt = data['지원대상']

In [ ]:
tpdt = data['지원내용']

In [ ]:
exdt.isnull().sum()

0

## 없애고 싶은 것:
* 홈페이지 주소
* 각종 특수문자
* 괄호 안에 있는 모든 것.
* 카테고리
* 한자 한국어로 바꾸기.

순서: 괄호 안에 있는 것들 -> 홈페이지(영어) -> 카테고리 ->한자 한국어로 바꾸기 -> 특수문자 제거

### 여러 개의 스페이스 바

### 괄호 안에 있는 것들

In [ ]:
exdt = exdt.str.replace('\n\n\n', '\n') # \n을 기준으로 문장 생성
exdt = exdt.str.replace('\n\n', '\n') # \n을 기준으로 문장 생성
exdt = exdt.str.replace('\n', '.') # \n을 기준으로 문장 생성
#exdt = exdt.str.replace('○', '') # 거슬리는 특수문자 제거
exdt = exdt +"."                  # 맨 마지막에 온점 찍기

In [ ]:
tpdt = tpdt.str.replace('\n\n\n', '\n') # \n을 기준으로 문장 생성
tpdt = tpdt.str.replace('\n\n', '\n') # \n을 기준으로 문장 생성
tpdt = tpdt.str.replace('\n', '.') # \n을 기준으로 문장 생성
#tpdt = tpdt.str.replace('○', '') # 거슬리는 특수문자 제거
tpdt = tpdt +"."                  # 맨 마지막에 온점 찍기

In [ ]:
tpdt[34]

'○ 항로표지 전문인력의 양성 및 교육ㆍ훈련에 관한 사항, 교육프로그램의 개발 및 보급.'

In [ ]:
tpdt[3]

'○ 자금용도 : 운영자금.○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부여(중견기업 최대 1,200억원, 중소기업 최대 600억원). ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여.○ 우대조건 : 최대 0.90%p 금리 우대, 심사절차 간소화 등.'

In [ ]:
exdt[88484]

'만3세~6세 어린이.저소득계층(최저생계비 200% 이내), 취학 전 아동.저소득층||일반...'

In [ ]:
import re
meat = re.sub(r'\([^)]*\)',"",exdt[88484])
meat

'만3세~6세 어린이.저소득계층, 취학 전 아동.저소득층||일반...'

### 영어 제거

In [ ]:
meat = meat + 'abcd'
meat

meat1 = re.sub(r'[!^a-zA-Z0-9]',"",meat)
meat1

'만세~세 어린이.저소득계층, 취학 전 아동.저소득층||일반.'

In [ ]:
# 지금까지 전처리 합친 코드

def preprocessing(x):
  meat = re.sub(r'\([^)]*\)',"",x)
  meat1 = re.sub(r'[!^a-zA-Z]',"",meat)
  return meat1.split(".")


In [ ]:
lala = preprocessing(tpdt[3])
lala

['○ 자금용도 : 운영자금',
 '○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부여',
 ' ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여',
 '○ 우대조건 : 최대 0',
 '90% 금리 우대, 심사절차 간소화 등',
 '']

### 카테고리
카테고리는 ||, /, > 로 구분되어있어요.

카테고리 이름은 저 기호랑 띄어쓰기 없이 바로 연결되어있어요,

그리고 보통 카테고리는 \n\n 여러 개 뒤에 나오는 듯?

그리고 셀의 맨 마지막에 나와용! 그래서 그냥 셀마다 하나하나 전처리 하고 합칠게여.



In [ ]:
#스트링을 온점을 기준으로 분리 이후에 마지막 스트링에 저 세가지 기호 중 하나 있으면 삭제? 할게여.

meat2 = meat1.split(".")

In [ ]:
meat2

['만세~세 어린이', '저소득계층, 취학 전 아동', '저소득층||일반', '']

In [ ]:
#몇몇 것들이 뒤에 empty string이 몇 개 나와서 이거 다 없애고 그 마지막 카테고리 없앨게여.
#일단 엠트 스트링 없애기
meat3 = []
for i in range(len(meat2)):
  if len(meat2[i]) != 0:
    meat3.append(meat2[i])

meat3

['만세~세 어린이', '저소득계층, 취학 전 아동', '저소득층||일반']

In [ ]:
cat_string = ["||", "/", ">"]
for i in range(len(meat3)):
  for j in range(3):
    if cat_string[j] in meat3[-i]:
      del meat3[-i]
      continue

  
meat3

['만세~세 어린이', '저소득계층, 취학 전 아동']

In [ ]:
meat4 = ".".join(meat3)
meat4
meat4 = meat4+"."
meat4

'만세~세 어린이.저소득계층, 취학 전 아동.'

In [ ]:
def non_cat(x):
  meat3 = []
  for i in range(len(x)):
    if len(x[i]) != 0:
      meat3.append(x[i])
  cat_string = ["||", "/", ">"]
  for i in range(len(meat3)):
    for j in range(3):
      if cat_string[j] in meat3[-i]:
        del meat3[-i]
        continue
  meat4 = ".".join(meat3)
  meat4 = meat4 + '.'
  return meat4

non_cat(lala)

'○ 자금용도 : 운영자금.○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부여. ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여.○ 우대조건 : 최대 0.90% 금리 우대, 심사절차 간소화 등.'

In [ ]:
lala = non_cat(lala)

In [ ]:
lala

'○ 자금용도 : 운영자금.○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부여. ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여.○ 우대조건 : 최대 0.90% 금리 우대, 심사절차 간소화 등.'

### 한자를 한글로 변환

In [ ]:
!pip install hanja

     |████████████████████████████████| 120 kB 37.1 MB/s 
     |████████████████████████████████| 265 kB 49.8 MB/s 
     |████████████████████████████████| 213 kB 68.2 MB/s 
     |████████████████████████████████| 280 kB 78.1 MB/s 
     |████████████████████████████████| 213 kB 58.5 MB/s 
     |████████████████████████████████| 213 kB 77.2 MB/s 
     |████████████████████████████████| 253 kB 61.2 MB/s 
     |████████████████████████████████| 252 kB 55.6 MB/s 
     |████████████████████████████████| 252 kB 62.0 MB/s 
     |████████████████████████████████| 242 kB 65.1 MB/s 
     |████████████████████████████████| 242 kB 67.2 MB/s 
     |████████████████████████████████| 242 kB 65.0 MB/s 
     |████████████████████████████████| 229 kB 78.5 MB/s 
     |████████████████████████████████| 229 kB 81.6 MB/s 
INFO: pip is looking at multiple versions of pytest-cov to determine which version is compatible with other requirements. This could take a while.
  Created wheel for hanja: filename=hanja

In [ ]:
lala

'○ 자금용도 : 운영자금.○ 한도 : 기존 대출한도 外 일정범위내 특별한도 부여. ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여.○ 우대조건 : 최대 0.90% 금리 우대, 심사절차 간소화 등.'

In [ ]:
import hanja
lili = hanja.translate(lala, 'substitution')
lili

'○ 자금용도 : 운영자금.○ 한도 : 기존 대출한도 외 일정범위내 특별한도 부여. ※ 매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여.○ 우대조건 : 최대 0.90% 금리 우대, 심사절차 간소화 등.'

### 특수문자 제거

In [ ]:
text = re.sub('[-=+,#/\?:^$@○*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', lili)
text

' 자금용도  운영자금. 한도  기존 대출한도 외 일정범위내 특별한도 부여.  매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여. 우대조건  최대 0.90 금리 우대 심사절차 간소화 등.'

# 숫자 있는 전처리

## 이제 지금까지 했던 것들 다 합칠게요!

숫자는 없애는 게 좋을지, 놔두는 게 좋을지 모르겠어서 둘 다 일단 만들겠습니당.

In [ ]:
#각 열마다 전처리!

def preprocess_1(x):
  x = x.str.replace('\n\n\n', '\n') # \n을 기준으로 문장 생성
  x = x.str.replace('\n\n', '\n') # \n을 기준으로 문장 생성
  x = x.str.replace('\n', '.') # \n을 기준으로 문장 생성
  x = x +"."       
  return x



In [ ]:
#각 열마다 전처리!

#import re
#import hanja


def real_preprocess_num(x):
  meat = re.sub(r'\([^)]*\)',"",x) #괄호 안 문자들 다 제거
  meat1 = re.sub(r'[!^a-zA-Z]',"",meat) #영어 제거
  lala = meat1.split(".") #카테고리 제거
  meat3 = []
  for i in range(len(lala)):
    if len(lala[i]) != 0:
      meat3.append(lala[i])
  cat_string = ["||", "/", ">"]
  temp1 = []
  for i in range(len(meat3)):
    for j in range(len(cat_string)):
      if cat_string[j] not in meat3[i]:
        temp1.append(meat3[i])
  meat4 = ".".join(meat3)
  meat4 = meat4 + '.'
  lili = hanja.translate(meat4, 'substitution') #한자 한글로 변환
  text = re.sub('[^.0-9가-힣\s]', '', lili) #한글, 숫자, 공백 빼고 다 제거
  text = re.sub('[-=+,#/\?:^$@○*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',text) #혹시 모를 특수문자들 다 제거ㅕ
  return text


## input 전처리

In [ ]:
exdt = data['지원대상']
exdt = preprocess_1(exdt)
exdt = exdt.astype(str)
exdt = exdt.apply(lambda x: real_preprocess_num(x))
exdt[88484]

'만3세6세 어린이.저소득계층 취학 전 아동.저소득층일반.'

In [ ]:
exdt.isnull().sum()

0

In [ ]:
tpdt = data['지원내용']
tpdt = preprocess_1(tpdt)
tpdt = tpdt.astype(str)
tpdt = tpdt.apply(lambda x: real_preprocess_num(x))
tpdt[3]

' 자금용도  운영자금. 한도  기존 대출한도 외 일정범위내 특별한도 부여.  매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여. 우대조건  최대 0.90 금리 우대 심사절차 간소화 등.'

In [ ]:
tpdt.isnull().sum()

0

In [ ]:
hcdt = data['선정기준']
hcdt = preprocess_1(hcdt)
hcdt = hcdt.astype(str)
hcdt = hcdt.apply(lambda x: real_preprocess_num(x))
hcdt[4]

' 기관장비설비사업의 사업자 선정은 다음의 순위로 하되 동일 순위자의 세부 우선순위는 저마력 기관 대체 신청자 소형어선 어업인 생분해성 . 기관장비설비사업의 사업자 선정은 다음의 순위로 하되 동일 순위자의 세부 우선순위는 저마력 기관 대체 신청자 소형어선 어업인 생분해성 어구 시범사업 참여 어업인 어업인 후계자 국산기관 사용자 순으로 함.  다만 등의 경우 집어등을 작업등보다 우선으로 하며 동일 순위자의 우선순위는 소형어선 어업인 어업인 후계자 순으로 함.       대체기관의 단종 등 부득이한 경우 상위 마력을 적용할 수 있으며 그 허용범위는 10 이하로 한다.     유류절감장치 및  등을 설치하고자 하는 자.     육상용 기관을 해상용 디젤기관으로 대체하고자 하는 자.     사용연수가 많은 노후 기관을 해상용 디젤기관으로 대체하고자 하는 자.     무동력 어선에 신규로 해상용 디젤기관을 설치하고자 하는 자.     노후장비설비를 대체하고자 하는 자.     법령으로 설치가 의무화된 노후장비설비를 희망하는 자.     새로운 장비설비를 설치하고자 하는 자.     가솔린 기관을 대체하고자 하는 자. 단  순위보다 우선지원 불가하며 전체 보조금의 30를 넘을 수 없음 .       기타 지자체 자체실정에 따라 우선순위 추가 가능.  사업자 선정을 위하여 필요한 경우 자체심의 기구에 상정심의할 수 있으며 사업포기에 대비하여 예비사업자를 선정할 수 있음 .       단 추가모집 사업자 선정시 지자체 판단에 의거 심의생략 가능.'

In [ ]:
hcdt.isnull().sum()

0

In [ ]:
input_data = exdt +   tpdt   + hcdt

In [ ]:
input_data.isnull().sum()

0

In [ ]:
input_data[40000]

' 입양특례법상 허가를 받은 입양기관에 의해 같은 법의 요건과 절차를 갖춰 국내 입양한 가정.  입양아동이 만 16세가 될 때까지.입양가정.월 15만 원의 양육수당 지급.생활병역생활지원입양가정.입양특례법상 허가를 받은 입양기관에 의해 같은 법의 요건과 절차를 갖춰 국내 입양한 가정.'

## output 전처리

In [ ]:
y_label = data['서비스목적']
y_label = preprocess_1(y_label)
y_label = y_label.astype(str)
y_label = y_label.apply(lambda x: real_preprocess(x))
y_label[3]

'코로나19로 피해를 입은 중소중견기업 지원 프로그램.'

In [ ]:
y_label.isnull().sum()

0

## 데이터 합치기

In [ ]:
preprocessed_data_withnum.to_csv("/content/drive/MyDrive/프로젝트2-B팀/요약 관련/지오_전처리_숫자있.csv")

# 숫자 없는 걸로 전처리.

In [ ]:
#각 열마다 전처리!

#import re
#import hanja


def real_preprocess_no(x):
  meat = re.sub(r'\([^)]*\)',"",x) #괄호 안 문자들 다 제거
  meat1 = re.sub(r'[!^a-zA-Z]',"",meat) #영어 제거
  lala = meat1.split(".") #카테고리 제거
  meat3 = []
  for i in range(len(lala)):
    if len(lala[i]) != 0:
      meat3.append(lala[i])
  cat_string = ["||", "/", ">"]
  temp1 = []
  for i in range(len(meat3)):
    for j in range(len(cat_string)):
      if cat_string[j] not in meat3[i]:
        temp1.append(meat3[i])
  meat4 = ".".join(meat3)
  meat4 = meat4 + '.'
  lili = hanja.translate(meat4, 'substitution') #한자 한글로 변환
  text = re.sub('[^.가-힣\s]', '', lili) #한글, 숫자, 공백 빼고 다 제거
  text = re.sub('[-=+,#/\?:^$@○*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',text) #혹시 모를 특수문자들 다 제거ㅕ
  return text


### input 전처리

In [ ]:
exdt1 = data['지원대상']
exdt1 = preprocess_1(exdt1)
exdt1 = exdt1.astype(str)
exdt1 = exdt1.apply(lambda x: real_preprocess_no(x))
exdt1[88484]

'만세세 어린이.저소득계층 취학 전 아동.저소득층일반.'

In [ ]:
tpdt1 = data['지원내용']
tpdt1 = preprocess_1(tpdt1)
tpdt1 = tpdt1.astype(str)
tpdt1 = tpdt1.apply(lambda x: real_preprocess_no(x))
tpdt1[3]

' 자금용도  운영자금. 한도  기존 대출한도 외 일정범위내 특별한도 부여.  매출액 등에 따른 기존 운영자금 대출한도 외 추가한도 부여. 우대조건  최대 . 금리 우대 심사절차 간소화 등.'

In [ ]:
hcdt1 = data['선정기준']
hcdt1 = preprocess_1(hcdt1)
hcdt1 = hcdt1.astype(str)
hcdt1 = hcdt1.apply(lambda x: real_preprocess_no(x))
hcdt1[4]

' 기관장비설비사업의 사업자 선정은 다음의 순위로 하되 동일 순위자의 세부 우선순위는 저마력 기관 대체 신청자 소형어선 어업인 생분해성 . 기관장비설비사업의 사업자 선정은 다음의 순위로 하되 동일 순위자의 세부 우선순위는 저마력 기관 대체 신청자 소형어선 어업인 생분해성 어구 시범사업 참여 어업인 어업인 후계자 국산기관 사용자 순으로 함.  다만 등의 경우 집어등을 작업등보다 우선으로 하며 동일 순위자의 우선순위는 소형어선 어업인 어업인 후계자 순으로 함.       대체기관의 단종 등 부득이한 경우 상위 마력을 적용할 수 있으며 그 허용범위는  이하로 한다.     유류절감장치 및  등을 설치하고자 하는 자.     육상용 기관을 해상용 디젤기관으로 대체하고자 하는 자.     사용연수가 많은 노후 기관을 해상용 디젤기관으로 대체하고자 하는 자.     무동력 어선에 신규로 해상용 디젤기관을 설치하고자 하는 자.     노후장비설비를 대체하고자 하는 자.     법령으로 설치가 의무화된 노후장비설비를 희망하는 자.     새로운 장비설비를 설치하고자 하는 자.     가솔린 기관을 대체하고자 하는 자. 단  순위보다 우선지원 불가하며 전체 보조금의 를 넘을 수 없음 .       기타 지자체 자체실정에 따라 우선순위 추가 가능.  사업자 선정을 위하여 필요한 경우 자체심의 기구에 상정심의할 수 있으며 사업포기에 대비하여 예비사업자를 선정할 수 있음 .       단 추가모집 사업자 선정시 지자체 판단에 의거 심의생략 가능.'

In [ ]:
input_data1 = exdt1 +  tpdt1 +   hcdt1

## output 전처리

In [ ]:
y_label1 = data['서비스목적']
y_label1 = preprocess_1(y_label1)
y_label1 = y_label1.astype(str)
y_label1 = y_label1.apply(lambda x: real_preprocess_no(x))
y_label1[3]

'코로나로 피해를 입은 중소중견기업 지원 프로그램.'

In [ ]:
preprocessed_data_wonum.to_csv("/content/drive/MyDrive/프로젝트2-B팀/요약 관련/지오_전처리_숫자없.csv")